## ETL PROCES REPORT
--------------------------------------------
### E: PULLING DATA FROM YELP'S API:

* Step 1: Get an API KEY  from yelp
* Investigate what kind of data to retrieve. In our case: General business profiles in the City of LA California
* The API returns only 50 results max per call
* Perform a test pull to look at the result and it's json structure to design the data frame

### T: PREPROCESSING THE JSON RESPONSE FROM THE API:
--------------------------------------------

* Declare/Initialize the variables/columns that will hold the downloaded data
* We build a dataset of 1000 resutls by making multiple calls dynamically and saving data into a dataframe on the fly
* Data processing consisted in extracting data from the json response and formatting it to create tabular data for the df
* The Transforming process included string manipulation regex and python scripting.

### L: LOADING TO SQL AND MONGODB:
--------------------------------------------
* This was the most straightforward step accomplished with no issues
* We simplify this proces by using sqlite instead of mysql or postgress
* We also saved csv files of the data before we loaded it into the database systems.


# Zip Code Data from the Census API (2017)

In [17]:
from census import Census
from us import states

from config import census_key
import numpy as np
import pandas as pd
import time

import json


In [3]:
c13 = Census(census_key, year=2013)

In [4]:
c17 = Census(census_key, year=2017)

# E: extraction

In [5]:
#data on all zip codes for 2013 and 2017 to test different results
_13census_data = c13.acs5.get(("B01003_001E", "B23025_005E"), {
                         'for': 'zip code tabulation area:*'})

_17census_data = c17.acs5.get(("B01003_001E", "B23025_005E"), {
                         'for': 'zip code tabulation area:*'})

## Test Results by comparing  2 different years

In [6]:
# Convert to DataFrame
_13_pd = pd.DataFrame(_13census_data)
_17_pd = pd.DataFrame(_17census_data)

In [7]:
_13_pd.head()

,B01003_001E,B23025_005E,zip code tabulation area
0,18450.0,1464.0,00601
1,41302.0,4767.0,00602
2,53683.0,4341.0,00603
3,6591.0,134.0,00606
4,28963.0,1134.0,00610


In [8]:
_17_pd.head()

,B01003_001E,B23025_005E,zip code tabulation area
0,17599.0,2454.0,00601
1,39209.0,2538.0,00602
2,50135.0,3588.0,00603
3,6304.0,204.0,00606
4,27590.0,1474.0,00610


# T: transform

In [9]:
# Column Reordering
_13_pd = _13_pd.rename(columns={"B01003_001E": "Population",
                                      "B23025_005E": "Unemployment Count",
                                      "zip code tabulation area": "Zipcode"})

# Column Reordering
_17_pd = _17_pd.rename(columns={"B01003_001E": "Population",
                                      "B23025_005E": "Unemployment Count",
                                      "zip code tabulation area": "Zipcode"})

In [10]:
_13_pd.head()

,Population,Unemployment Count,Zipcode
0,18450.0,1464.0,00601
1,41302.0,4767.0,00602
2,53683.0,4341.0,00603
3,6591.0,134.0,00606
4,28963.0,1134.0,00610


In [11]:
_17_pd.head()

,Population,Unemployment Count,Zipcode
0,17599.0,2454.0,00601
1,39209.0,2538.0,00602
2,50135.0,3588.0,00603
3,6304.0,204.0,00606
4,27590.0,1474.0,00610


In [12]:
# Add Employment Rate

_17_pd["Unemployment Rate"] = 100 * \
    _17_pd["Unemployment Count"].astype(
        int) / _17_pd["Population"].astype(int)

In [13]:
_17_pd.head()

,Population,Unemployment Count,Zipcode,Unemployment Rate
0,17599.0,2454.0,00601,13.943974
1,39209.0,2538.0,00602,6.473004
2,50135.0,3588.0,00603,7.156677
3,6304.0,204.0,00606,3.236041
4,27590.0,1474.0,00610,5.342515


In [14]:
_17_pd = _17_pd[["Zipcode", "Population","Unemployment Count", "Unemployment Rate"]]

In [15]:
_17_pd.head()

,Zipcode,Population,Unemployment Count,Unemployment Rate
0,00601,17599.0,2454.0,13.943974
1,00602,39209.0,2538.0,6.473004
2,00603,50135.0,3588.0,7.156677
3,00606,6304.0,204.0,3.236041
4,00610,27590.0,1474.0,5.342515


In [16]:
print('2013 rows: ',len(_13_pd))
print('2017 rows: ', len(_17_pd))
print()
_17_pd.head()

2013 rows:  33120
2017 rows:  33120



,Zipcode,Population,Unemployment Count,Unemployment Rate
0,00601,17599.0,2454.0,13.943974
1,00602,39209.0,2538.0,6.473004
2,00603,50135.0,3588.0,7.156677
3,00606,6304.0,204.0,3.236041
4,00610,27590.0,1474.0,5.342515


# L:  load
-----------------------------------------------
* Save df to csv format

In [18]:
_17_pd.to_csv("Resources/censusZips2017.csv", index=False)


* load dataframe into mongo

In [22]:
import pymongo

In [52]:
# Setup connection to mongodb
conn = "mongodb://localhost:27017"
client = pymongo.MongoClient(conn)

In [23]:
# Select database and collection to use(Census)
db = client.us_census_zips
collection = db.zips_2017

### Yelp Data: Convert  dataframe to dictionary format for mongodb L oading

In [ ]:
# Yelp collection
collection2 = db.yelp_biz_profiles

In [55]:
yelpDict = yelp_data.to_dict(orient='records')

In [58]:
collection2.insert_many(yelpDict)

* Test query database

In [59]:
results2 = collection2.find()

In [60]:
results2

In [61]:
for result in results2[0:2]:
    print()
    print(result)



{'_id': ObjectId('5ce04c56be1a7737c64894ab'), 'id': 'TkFEKhsCixPWlShULKvMdQ', 'alias': 'bottega-louie-los-angeles', 'name': 'Bottega Louie', 'categories': 'Italian, Bakeries, Breakfast & Brunch', 'review_count': '16243', 'rating': '4.0', 'lat': '34.0469300995766', 'long': '-118.256601457672', 'address': '700 S Grand Ave Los Angeles, CA 90017', 'city': 'Los Angeles', 'Zipcode': '90017', 'country': 'US', 'state': 'CA', 'phone': '+12138021470'}

{'_id': ObjectId('5ce04c56be1a7737c64894ac'), 'id': '7O1ORGY36A-2aIENyaJWPg', 'alias': 'howlin-rays-los-angeles-3', 'name': "Howlin' Ray's", 'categories': 'Southern, Chicken Shop, American (Traditional)', 'review_count': '5037', 'rating': '4.5', 'lat': '34.0614861063899', 'long': '-118.239554800093', 'address': '727 N Broadway Ste 128 Los Angeles, CA 90012', 'city': 'Los Angeles', 'Zipcode': '90012', 'country': 'US', 'state': 'CA', 'phone': '+12139358399'}


### Census Data: Convert  dataframe to dictionary format for mongodb L oading

In [24]:
dictData = _17_pd.to_dict(orient='records')

In [25]:
collection.insert_many(dictData)

* Test query database

In [26]:
results = collection.find()

In [27]:
type(results)

pymongo.cursor.Cursor

In [28]:
for result in results[0:2]:
    print()
    print(result)



{'_id': ObjectId('5ce007c6be1a7737c648134a'), 'Zipcode': '00601', 'Population': 17599.0, 'Unemployment Count': 2454.0, 'Unemployment Rate': 13.9439740894369}

{'_id': ObjectId('5ce007c6be1a7737c648134b'), 'Zipcode': '00602', 'Population': 39209.0, 'Unemployment Count': 2538.0, 'Unemployment Rate': 6.473003647121835}


* First document in the collection

In [29]:
collection.find()[0]

{'_id': ObjectId('5ce007c6be1a7737c648134a'),
 'Zipcode': '00601',
 'Population': 17599.0,
 'Unemployment Count': 2454.0,
 'Unemployment Rate': 13.9439740894369}

### Example Using  SQL database from  Dataframe: 1 table

In [30]:
from sqlalchemy import create_engine

In [31]:
database_path = "Resources/us_census.sqlite"

In [32]:
engine = create_engine(f"sqlite:///{database_path}")

In [33]:
engine.table_names()

[]

In [34]:
_17_pd.to_sql(name='us_census_zips', con=engine, if_exists='append', index=False)

In [35]:
engine.table_names()

['us_census_zips']

### Test Query the biz_profile table

In [37]:
pd.read_sql_query('select * from us_census_zips', con=engine).head()

,Zipcode,Population,Unemployment Count,Unemployment Rate
0,00601,17599.0,2454.0,13.943974
1,00602,39209.0,2538.0,6.473004
2,00603,50135.0,3588.0,7.156677
3,00606,6304.0,204.0,3.236041
4,00610,27590.0,1474.0,5.342515


## Combine Yelp and Census Data

In [41]:
yelp_data = pd.read_csv(
    "Resources/la_yelp_1000.csv", dtype="object", encoding="utf-8")

In [42]:
yelp_data.head()

,id,alias,name,categories,review_count,rating,lat,long,address,city,zip_code,country,state,phone
0,TkFEKhsCixPWlShULKvMdQ,bottega-louie-los-angeles,Bottega Louie,"Italian, Bakeries, Breakfast & Brunch",16243,4.0,34.0469300995766,-118.256601457672,"700 S Grand Ave Los Angeles, CA 90017",Los Angeles,90017,US,CA,+12138021470
1,7O1ORGY36A-2aIENyaJWPg,howlin-rays-los-angeles-3,Howlin' Ray's,"Southern, Chicken Shop, American (Traditional)",5037,4.5,34.0614861063899,-118.239554800093,"727 N Broadway Ste 128 Los Angeles, CA 90012",Los Angeles,90012,US,CA,+12139358399
2,KQBGm5G8IDkE8LeNY45mbA,wurstküche-los-angeles-2,Wurstküche,"Hot Dogs, German, Gastropubs",8058,4.0,34.0456047058105,-118.236061096191,"800 E 3rd St Los Angeles, CA 90013",Los Angeles,90013,US,CA,+12136874444
3,iSZpZgVnASwEmlq0DORY2A,daikokuya-little-tokyo-los-angeles,Daikokuya Little Tokyo,"Ramen, Noodles",8122,4.0,34.05008090944,-118.2401804513,"327 E 1st St Los Angeles, CA 90012",Los Angeles,90012,US,CA,+12136261680
4,DJek3FUewBzMc0gS-Gms9w,the-morrison-los-angeles,The Morrison,"Gastropubs, Burgers, Bars",4072,4.5,34.12384,-118.26868,"3179 Los Feliz Blvd Los Angeles, CA 90039",Los Angeles,90039,US,CA,+13236671839


In [46]:
yelp_data.rename(columns={"zip_code": "Zipcode"}, inplace=True)

In [47]:
yelp_data.head()

,id,alias,name,categories,review_count,rating,lat,long,address,city,Zipcode,country,state,phone
0,TkFEKhsCixPWlShULKvMdQ,bottega-louie-los-angeles,Bottega Louie,"Italian, Bakeries, Breakfast & Brunch",16243,4.0,34.0469300995766,-118.256601457672,"700 S Grand Ave Los Angeles, CA 90017",Los Angeles,90017,US,CA,+12138021470
1,7O1ORGY36A-2aIENyaJWPg,howlin-rays-los-angeles-3,Howlin' Ray's,"Southern, Chicken Shop, American (Traditional)",5037,4.5,34.0614861063899,-118.239554800093,"727 N Broadway Ste 128 Los Angeles, CA 90012",Los Angeles,90012,US,CA,+12139358399
2,KQBGm5G8IDkE8LeNY45mbA,wurstküche-los-angeles-2,Wurstküche,"Hot Dogs, German, Gastropubs",8058,4.0,34.0456047058105,-118.236061096191,"800 E 3rd St Los Angeles, CA 90013",Los Angeles,90013,US,CA,+12136874444
3,iSZpZgVnASwEmlq0DORY2A,daikokuya-little-tokyo-los-angeles,Daikokuya Little Tokyo,"Ramen, Noodles",8122,4.0,34.05008090944,-118.2401804513,"327 E 1st St Los Angeles, CA 90012",Los Angeles,90012,US,CA,+12136261680
4,DJek3FUewBzMc0gS-Gms9w,the-morrison-los-angeles,The Morrison,"Gastropubs, Burgers, Bars",4072,4.5,34.12384,-118.26868,"3179 Los Feliz Blvd Los Angeles, CA 90039",Los Angeles,90039,US,CA,+13236671839


## Merge the two data sets on zip code

In [48]:
yelp_census_combined = pd.merge(
    yelp_data, _17_pd, how="left", on=["Zipcode", "Zipcode"])

In [49]:
yelp_census_combined.head()

,id,alias,name,categories,review_count,rating,lat,long,address,city,Zipcode,country,state,phone,Population,Unemployment Count,Unemployment Rate
0,TkFEKhsCixPWlShULKvMdQ,bottega-louie-los-angeles,Bottega Louie,"Italian, Bakeries, Breakfast & Brunch",16243,4.0,34.0469300995766,-118.256601457672,"700 S Grand Ave Los Angeles, CA 90017",Los Angeles,90017,US,CA,+12138021470,26390.0,920.0,3.486169
1,7O1ORGY36A-2aIENyaJWPg,howlin-rays-los-angeles-3,Howlin' Ray's,"Southern, Chicken Shop, American (Traditional)",5037,4.5,34.0614861063899,-118.239554800093,"727 N Broadway Ste 128 Los Angeles, CA 90012",Los Angeles,90012,US,CA,+12139358399,33783.0,973.0,2.880147
2,KQBGm5G8IDkE8LeNY45mbA,wurstküche-los-angeles-2,Wurstküche,"Hot Dogs, German, Gastropubs",8058,4.0,34.0456047058105,-118.236061096191,"800 E 3rd St Los Angeles, CA 90013",Los Angeles,90013,US,CA,+12136874444,11957.0,817.0,6.832818
3,iSZpZgVnASwEmlq0DORY2A,daikokuya-little-tokyo-los-angeles,Daikokuya Little Tokyo,"Ramen, Noodles",8122,4.0,34.05008090944,-118.2401804513,"327 E 1st St Los Angeles, CA 90012",Los Angeles,90012,US,CA,+12136261680,33783.0,973.0,2.880147
4,DJek3FUewBzMc0gS-Gms9w,the-morrison-los-angeles,The Morrison,"Gastropubs, Burgers, Bars",4072,4.5,34.12384,-118.26868,"3179 Los Feliz Blvd Los Angeles, CA 90039",Los Angeles,90039,US,CA,+13236671839,29092.0,1299.0,4.465145


# Save the Combined Data Frame as a csv

In [50]:
yelp_census_combined.to_csv(
    "Resources/yelp_census_data.csv", encoding="utf-8", index=False)

In [51]:
yelp_census_combined.head()

,id,alias,name,categories,review_count,rating,lat,long,address,city,Zipcode,country,state,phone,Population,Unemployment Count,Unemployment Rate
0,TkFEKhsCixPWlShULKvMdQ,bottega-louie-los-angeles,Bottega Louie,"Italian, Bakeries, Breakfast & Brunch",16243,4.0,34.0469300995766,-118.256601457672,"700 S Grand Ave Los Angeles, CA 90017",Los Angeles,90017,US,CA,+12138021470,26390.0,920.0,3.486169
1,7O1ORGY36A-2aIENyaJWPg,howlin-rays-los-angeles-3,Howlin' Ray's,"Southern, Chicken Shop, American (Traditional)",5037,4.5,34.0614861063899,-118.239554800093,"727 N Broadway Ste 128 Los Angeles, CA 90012",Los Angeles,90012,US,CA,+12139358399,33783.0,973.0,2.880147
2,KQBGm5G8IDkE8LeNY45mbA,wurstküche-los-angeles-2,Wurstküche,"Hot Dogs, German, Gastropubs",8058,4.0,34.0456047058105,-118.236061096191,"800 E 3rd St Los Angeles, CA 90013",Los Angeles,90013,US,CA,+12136874444,11957.0,817.0,6.832818
3,iSZpZgVnASwEmlq0DORY2A,daikokuya-little-tokyo-los-angeles,Daikokuya Little Tokyo,"Ramen, Noodles",8122,4.0,34.05008090944,-118.2401804513,"327 E 1st St Los Angeles, CA 90012",Los Angeles,90012,US,CA,+12136261680,33783.0,973.0,2.880147
4,DJek3FUewBzMc0gS-Gms9w,the-morrison-los-angeles,The Morrison,"Gastropubs, Burgers, Bars",4072,4.5,34.12384,-118.26868,"3179 Los Feliz Blvd Los Angeles, CA 90039",Los Angeles,90039,US,CA,+13236671839,29092.0,1299.0,4.465145
